# HSI Library チュートリアル
### 目次
1. indian-pinesをダウンロード
2. ライブラリをダウンロード
3. ライブラリを利用したバンド削減
4. DeepLearningモデルの訓練

### 1. indian-pinesをダウンロード
まずチュートリアルに使用するindian-pinesのデータをダウンロードします。

In [20]:
# 保存先を指定
WORKDIR='./data'

# データのダウンロード
!wget https://www.ehu.eus/ccwintco/uploads/2/22/Indian_pines.mat -P $WORKDIR/indian_pines/ # data
!wget https://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat -P $WORKDIR/indian_pines/ # ground truth

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/masaya/.wget-hsts'. HSTS will be disabled.
--2024-09-23 14:16:37--  https://www.ehu.eus/ccwintco/uploads/2/22/Indian_pines.mat
www.ehu.eus (www.ehu.eus) をDNSに問いあわせています... 158.227.0.65, 2001:720:1410::65
www.ehu.eus (www.ehu.eus)|158.227.0.65|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 6296374 (6.0M)
‘./data/indian_pines/Indian_pines.mat.1’ に保存中

Indian_pines.mat.1    0%[                    ]       0  --.-KB/s               ^C
Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/masaya/.wget-hsts'. HSTS will be disabled.
--2024-09-23 14:16:39--  https://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat
www.ehu.eus (www.ehu.eus) をDNSに問いあわせています... 158.227.0.65, 2001:720:1410::65
www.ehu.eus (www.ehu.eus)|158.227.0.65|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しま

.matファイルでダウンロードされたデータを、pythonで扱いやすい.npyファイルへ変換します

In [31]:
import glob
import os
import pathlib
import numpy as np
from scipy.io import loadmat

def mat2npy(data_path):
    print(data_path)
    local, ext = os.path.splitext(data_path)
    if ext != ".mat":
        print(f'{ext=}')
        return
    
    data = loadmat(data_path)
    print(data_path)
    for key in data:
        if isinstance(data[key], np.ndarray):
            print(data[key].shape)
            print(np.unique(data[key]))
            save_path = os.path.join(
                local,
            )
            np.save(save_path, data[key]) # npyに変換して保存

# データのロード
data_paths = [f'{WORKDIR}/indian_pines/Indian_pines.mat', f'{WORKDIR}/indian_pines/indian_pines_gt.mat']
for data in data_paths:
    mat2npy(data) 
            

./data/indian_pines/Indian_pines_gt.mat
./data/indian_pines/Indian_pines_gt.mat
(145, 145)
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
./data/indian_pines/Indian_pines.mat
./data/indian_pines/Indian_pines.mat
(145, 145, 220)
[ 955  961  962 ... 9279 9360 9604]


In [23]:
!pwd

/home/masaya/in-spector-project/HSI-library


### 2.ライブラリをダウンロード
続いてライブラリをインストールします。

In [ ]:
# ryeを利用している場合
!rye add hsi_feature_extraction --git https://github.com/SuperHotDogCat/HSI-library.git 

In [ ]:
# pipの場合
%pip install https://github.com/SuperHotDogCat/HSI-library.git 

### 3. ライブラリを利用したバンド削減
実際にライブラリを使用して、HSIのバンド数を削減するアルゴリズムを実行します。
ここでは、セレクションメソッドの一つである**VIF**を例に上げて説明します。

In [33]:
# データのロード
data_path = f'{WORKDIR}/indian_pines/Indian_pines.npy'
label_path = f'{WORKDIR}/indian_pines/Indian_pines_gt.npy'
data = np.load(data_path)[np.newaxis,:32,:32,:]
print(data.shape)

(1, 32, 32, 220)


In [34]:
# ライブラリからVIFExtractorをインポート
from hsi_feature_extraction.selection.vif import VIFExtractor
processor = VIFExtractor()

# アルゴリズムの適用
processor.fit(data)

TypeError: Can't instantiate abstract class VIFExtractor with abstract method get_num_channels